In [50]:
from pandas import read_csv
from random import seed
from random import randrange

In [41]:
# Load dataset from UCI archive and describe it
data = read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt', header=None)
data.describe()
dataset = data.values.tolist()  
    

In [42]:
# Gini Index: the cost function used to evaluate splits in the dataset. The score rates how good a split 
# is (0 is good 0.5 (50/50 for binary classifications)). The function groups the data and calculates the proportionality 
# of the split, "gini" is then calculated for each node: 1-sum(proportion * proportion). It is then weighted by the group
# size: gini = (1.0 - sum(proportion * proportion)) * (group_size/total_samples). These scores are then added across all
# child nodes of the split.
def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    
    for group in groups:
        size = float(len(group))
        # divide by zero issue
        if size == 0:
            continue
        score = 0.0
        
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    
    return gini

In [43]:
# Write some code to split the data by some arbitrary value
def test_split(index, value, dataset):
    left, right = list(), list()
    
    for row in dataset:
        
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    
    return left, right

In [44]:
# Find the best split point by iterating over each feature and then each value for that feature
#def get_split(dataset):
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    for index in range(len(dataset[0])-1):
        
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


In [45]:
# Build the (whole) tree: We will develop the first node by doing our test on the dataset. There are 3 parts:
# (1) Terminal Nodes
# (2) Recursive Splitting
# (3) Building the tree

# (1) Terminal Nodes: When do we stop our tree from growing? "Max Tree Depth" (Vertical Limit) & "Min Node Records" (Min training per node)
# Both of these limits help reduce overfitting. We can define the terminal node as the node where all of one class belongs to one group

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    
    return max(set(outcomes), key=outcomes.count)

# (2) Recursive Splitting: In this part we will call the split function repeatedly to generate nodes. We will create a function
# that takes a node, the two limits from above and the current depth of the node. 

def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups']) #Now that it is partitioned
    
    # Check for terminality (sounds like the right word)
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return 
    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # Left child node
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # Right child node
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        
    return None

# (3) Build a tree: Cobble it all together.

def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
     
    return root


In [46]:
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print(f"""{depth*' '}[{node}]""")

# tree = build_tree(dataset, 2, 1)
# print_tree(tree)


In [47]:
# Let's use this to make a prediction
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
    return None

# # Test on an arbitrary dataset
# dataset = [[2.771244718,1.784783929,0],
#         [1.728571309,1.169761413,0],
#         [3.678319846,2.81281357,0],
#         [3.961043357,2.61995032,0],
#         [2.999208922,2.209014212,0],
#         [7.497545867,3.162953546,1],
#         [9.00220326,3.339047188,1],
#         [7.444542326,0.476683375,1],
#         [10.12493903,3.234550982,1],
#         [6.642287351,3.319983761,1]]

# #  predict with a stump
# stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
# for row in dataset:
#     prediction = predict(stump, row)
#     print('Expected=%d, Got=%d' % (row[-1], prediction))

In [49]:
# The Real Deal, train and test on the actual data.
# We will use kfold cv = 5, a new func decision_tree is created for management purposes
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    
    for i in range(n_folds):
        fold = list()
        
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        
        dataset_split.append(fold)
    
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    
    for i in range(len(actual)):
        
        if actual[i] == predicted[i]:
            correct += 1
            
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    
    return scores

def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print(f'Scores: {scores}')
print(f'Mean Accuracy: {(sum(scores)/float(len(scores)))}')

Scores: [95.25547445255475, 98.90510948905109, 94.8905109489051, 95.98540145985402, 97.44525547445255]
Mean Accuracy: 96.4963503649635


In [ ]:
""" RECAP:
In this module we covered creating a decision tree algorithm from scratch. We covered how to split the data and how to 
recursively build a decision tree from these splits. Then we followed up by implementing what we learned using the Kaggle 
dataset for bank_note_authentication, a binary classification problem. This would work well with other datasets such as 
the Kaggle set for breast cancer diagnosis, or anything that has a binary output. This algo is great for datasets with high
dimensionality, and a binary output. However in this case it has not been tuned and that may be something to touch on in
the future. 
"""